# Group 5K fold

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, average_precision_score
)
from imblearn.over_sampling import SMOTE
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
# CSV 
file_path = '/Users/bg.lim/Downloads/TAMU_Agri/New_CGM/filtered_thre.csv'

# 
df = pd.read_csv(file_path, encoding="utf-8")

# Convert Time_24h to float hour
df["Time_24h"] = pd.to_datetime(df["Time_24h"], format="%H:%M", errors="coerce")
df["Time_24h"] = df["Time_24h"].dt.hour + df["Time_24h"].dt.minute / 60

X_columns = [
    "15m_G_Diff_5", "15m_G_Diff_6","15m_G_Diff_7","15m_G_Diff_8",
    "15m_G_Diff_9","15m_G_Diff_10",
    "Z_Previous_5","Z_Previous_6","Z_Previous_7","Z_Previous_8","Z_Previous_9","Z_Previous_10",
    "Time_24h", "hungry_weighted", "EMA_T_Diff",
    "mean_intensity_1h", "mean_intensity_2h", "mean_intensity_3h",
    "HR_mean_1h", "HR_mean_2h", "HR_mean_3h",
    "HR_std_1h", "HR_std_2h", "HR_std_3h",
    "HR_slope_1h", "HR_slope_2h", "HR_slope_3h",
    "TimeInBed", "Efficiency", "Sleep_Diff",
    "TimeInBed_isnull", "Efficiency_isnull", "Sleep_Diff_isnull",
    "hungry_weighted_isnull", "bored_weighted_isnull", "How_stressed_weighted_isnull",
    "How_anxious_weighted_isnull", "How_tired_weighted_isnull",
    "Glucose_range_pre_1_2.5h","Glucose_spread_ratio_pre_1_2.5h",
    "Glucose_skew_hint_pre_1_2.5h","Glucose_std_pre_1_2.5h",
    "Glucose_iqr_to_std_pre_1_2.5h",    
    "1_1.5h_pre_mean", "1_1.5h_pre_std", "1_1.5h_pre_slope",
    "1.5_2h_pre_mean", "1.5_2h_pre_std", "1.5_2h_pre_slope",
    "2_2.5h_pre_mean", "2_2.5h_pre_std", "2_2.5h_pre_slope",
    "Glucose_q25_pre_1_2.5h","Z_Previous_4","Z_HR", "Z_Intensity",
    "HR","Intensity", "1h_pre_Thre", "Glucose_pre_1h", "Glucose_pre_1.5h",
    "Glucose_pre_2h", "G_minus_T_pre_1.5h", "G_minus_T_pre_2h",
    "Threshold_pre_2h", "Threshold_pre_1.5h"
    
]


# Remove rows with missing values
df_model = df[X_columns + ["GE_1h", "StudyID"]].dropna()

# 기본 데이터셋 준비 (기존과 동일)
df_model = df[X_columns + ["GE_1h", "StudyID"]].dropna()
X = df_model[X_columns]
y = df_model["GE_1h"].astype(int)
groups = df_model["StudyID"]

# GroupKFold 설정
gkf = GroupKFold(n_splits=5)

# 결과 저장용 리스트
metrics = []
feature_importance_list = []

threshold = 0.3  # 고정 threshold

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    print(f"\n📂 Fold {fold + 1}")

    X_train_raw, y_train_raw = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    # SMOTE 적용
    minority_class_size = np.bincount(y_train_raw)[1]
    k_neighbors = min(5, minority_class_size - 1) if minority_class_size > 1 else 1
    smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
    X_train, y_train = smote.fit_resample(X_train_raw, y_train_raw)

    # 모델 학습
    model = RandomForestClassifier(
        n_estimators=1000,
        min_samples_split=50,
        min_samples_leaf=50,
        max_features=None,
        max_depth=50,
        criterion='entropy',
        class_weight=None,
        bootstrap=True,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # 예측 및 평가
    y_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)

    metrics.append({
        "Accuracy": accuracy_score(y_val, y_pred),
        "Precision": precision_score(y_val, y_pred, zero_division=0),
        "Recall": recall_score(y_val, y_pred, zero_division=0),
        "F1": f1_score(y_val, y_pred, zero_division=0),
        "ROC AUC": roc_auc_score(y_val, y_proba),
        "PR AUC": average_precision_score(y_val, y_proba)
    })

    # Feature importance 저장
    feature_importance_list.append(model.feature_importances_)

# 평균 결과 출력
results_df = pd.DataFrame(metrics)
print("\n📊 GroupKFold (5-fold) 평균 성능:\n")
print(results_df.mean().round(4))

# Feature importance 평균
feature_importance_avg = np.mean(feature_importance_list, axis=0)
importances = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importance_avg
}).sort_values("Importance", ascending=False)

print("\n🔍 평균 Feature Importance (Top 15):")
print(importances.head(15))


/var/folders/r0/h30ywvr55x98xwjf49klh76m0000gp/T/ipykernel_38918/3566521292.py:17: DtypeWarning: Columns (102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding="utf-8")



📂 Fold 1

📂 Fold 2

📂 Fold 3

📂 Fold 4

📂 Fold 5

📊 GroupKFold (5-fold) 평균 성능:

Accuracy     0.7071
Precision    0.1533
Recall       0.4498
F1           0.2277
ROC AUC      0.6596
PR AUC       0.1470
dtype: float64

🔍 평균 Feature Importance (Top 15):
               Feature  Importance
57           Intensity    0.236737
15   mean_intensity_1h    0.119838
13     hungry_weighted    0.057979
12            Time_24h    0.054551
0         15m_G_Diff_5    0.040470
50      2_2.5h_pre_std    0.036418
44      1_1.5h_pre_std    0.034187
47      1.5_2h_pre_std    0.027030
1         15m_G_Diff_6    0.025320
45    1_1.5h_pre_slope    0.024274
2         15m_G_Diff_7    0.023953
53        Z_Previous_4    0.021543
29          Sleep_Diff    0.019442
27           TimeInBed    0.017898
62  G_minus_T_pre_1.5h    0.017619


In [5]:
import joblib

# 모델 저장
joblib.dump(model, "rf_model_last_fold.pkl")

# 데이터 저장
df_model.to_pickle("df_model.pkl")


In [7]:
# y_proba는 마지막 fold 결과를 사용
tp_mask = (y_pred == 1) & (y_val == 1)
tp_indices = y_val.index[tp_mask]

tp_enriched = df_model.loc[tp_indices].copy()
print(f"TP events: {len(tp_enriched)}")


TP events: 281


In [71]:
def summarize_alerts_table(alerts_df: pd.DataFrame):
    if alerts_df is None or alerts_df.empty:
        print("No alerts after policy.")
        empty = pd.DataFrame(columns=["StudyID","UniqueDays","TotalAlerts","Alerts_per_day_mean","Alerts_per_day_median"])
        return empty, np.nan

    per_pid_day = alerts_df.groupby(["StudyID","date"]).size().reset_index(name="alerts")
    per_pid = (per_pid_day.groupby("StudyID")["alerts"]
               .agg(UniqueDays="count", TotalAlerts="sum",
                    Alerts_per_day_mean="mean", Alerts_per_day_median="median")
               .reset_index())

    success_rate = float(alerts_df["success"].mean())

    print("\n=== Policy-level alert summary (TABLE) ===")
    print(f"Total alerts: {len(alerts_df)}")
    print(f"Overall success rate: {success_rate:.3f}")
    print(f"Median alerts/day across participants: {per_pid['Alerts_per_day_median'].median():.2f}")
    print(f"Mean alerts/day across participants: {per_pid['Alerts_per_day_mean'].mean():.2f}")

    return per_pid, success_rate


In [99]:
def generate_alerts_with_policy_fast(
    pdf: pd.DataFrame,
    threshold: float = 0.3,
    min_hour: float = 8.0,
    max_hour: float = 22.0,
    require_two_hits: bool = True,
    two_hit_window_min: int = 30,
    cooldown_min: int = 120,
    per_day_cap: int = 4,
    high_conf_single_hit_prob: float = None  # NEW: e.g., 0.60
):
    d = pdf.loc[(pdf["hour_float"] >= min_hour) & (pdf["hour_float"] <= max_hour),
                ["StudyID","ts","date","y_true","y_prob"]].copy()
    d["is_pos"] = (d["y_prob"] >= threshold).astype(np.uint8)
    d = d[d["is_pos"] == 1]
    if d.empty:
        return pd.DataFrame(columns=["StudyID","ts","date","y_true_at_alert","y_prob_at_alert","success"])

    alerts = []

    for (pid, day), g in d.sort_values(["StudyID","date","ts"]).groupby(["StudyID","date"]):
        times = g["ts"].to_numpy()                      # numpy.datetime64[ns]
        probs = g["y_prob"].to_numpy(dtype=float)
        yts   = g["y_true"].to_numpy(dtype=int)

        # 후보 인덱스(두-hit or 고확률 단발)
        if require_two_hits:
            win = np.timedelta64(two_hit_window_min, "m")
            j_right = np.searchsorted(times, times + win, side="right") - 1
            has_pair = j_right > np.arange(len(times))
            has_pair &= (times[j_right] - times) <= win

            if high_conf_single_hit_prob is not None:
                high_conf = probs >= float(high_conf_single_hit_prob)
                cand_mask = has_pair | high_conf
            else:
                cand_mask = has_pair

            if not np.any(cand_mask):
                continue
            cand_idx = np.nonzero(cand_mask)[0]
        else:
            cand_idx = np.arange(len(times))

        selected = []
        if len(cand_idx) > 0:
            cooldown = np.timedelta64(cooldown_min, "m")
            # TZ 경고 없는 안전한 초기화: 첫 이벤트보다 충분히 과거
            last_alert_time = times[0] - cooldown - np.timedelta64(1, "m")

            # 확률 높은 순으로 그리디 선택
            order = np.argsort(-probs[cand_idx])
            for k in order:
                i = cand_idx[k]
                if times[i] - last_alert_time < cooldown:
                    continue
                selected.append(i)
                last_alert_time = times[i]
                if len(selected) >= per_day_cap:
                    break

        if not selected:
            continue

        sel = np.array(selected, dtype=int)
        alerts.extend([
            {"StudyID": pid, "ts": times[i], "date": day,
             "y_true_at_alert": int(yts[i]), "y_prob_at_alert": float(probs[i])}
            for i in sel
        ])

    if not alerts:
        return pd.DataFrame(columns=["StudyID","ts","date","y_true_at_alert","y_prob_at_alert","success"])

    alerts_df = pd.DataFrame(alerts)
    alerts_df["success"] = alerts_df["y_true_at_alert"].astype(int)
    return alerts_df

S1 = dict(
    threshold=0.30,
    min_hour=8.0, max_hour=22.0,
    require_two_hits=True,
    two_hit_window_min=45,
    cooldown_min=90,
    per_day_cap=5,
    high_conf_single_hit_prob=0.60
)
alerts_S1 = generate_alerts_with_policy_fast(pred_df, **S1)
per_pid_S1, sr_S1 = summarize_alerts_table(alerts_S1)

tp = int(alerts_S1["success"].sum())
total = len(alerts_S1)
print(f"Event-level: TP={tp}, FP={total - tp}, Total={total} (SR={tp/total:.3f})")



=== Policy-level alert summary (TABLE) ===
Total alerts: 632
Overall success rate: 0.131
Median alerts/day across participants: 1.50
Mean alerts/day across participants: 1.71
Event-level: TP=83, FP=549, Total=632 (SR=0.131)


In [119]:
import numpy as np
import pandas as pd

# --- 0) S1 policy (exactly as you specified) ---
S1 = dict(
    threshold=0.30,
    min_hour=8.0, max_hour=22.0,
    require_two_hits=True,
    two_hit_window_min=45,
    cooldown_min=90,
    per_day_cap=5,
    high_conf_single_hit_prob=0.60
)

# --- 1) ML alerts using S1 (no retraining) ---
alerts_S1 = generate_alerts_with_policy_fast(pred_df, **S1)

# Use the 2-return table version to avoid prior name collisions
per_pid_S1, sr_S1 = summarize_alerts_table(alerts_S1)

tp = int(alerts_S1["success"].sum())
total = len(alerts_S1)
print(f"\n[ML/S1] Event-level: TP={tp}, FP={total - tp}, Total={total} (SR={tp/total:.3f})")

# If you also keep a dict-style summarize_alerts, this gives a compact summary dict:
def summarize_alerts_dict(alerts_df: pd.DataFrame, name=""):
    if alerts_df is None or alerts_df.empty:
        return dict(name=name, total_alerts=0, success_rate=np.nan,
                    median_alerts_per_day=np.nan, mean_alerts_per_day=np.nan)
    per_pid_day = alerts_df.groupby(["StudyID","date"]).size().reset_index(name="alerts")
    per_pid = (per_pid_day.groupby("StudyID")["alerts"]
               .agg(["count","sum","mean","median"]).reset_index()
               .rename(columns={"count":"UniqueDays","sum":"TotalAlerts",
                                "mean":"Alerts_per_day_mean","median":"Alerts_per_day_median"}))
    sr = float(alerts_df["success"].mean())
    return dict(
        name=name,
        total_alerts=int(len(alerts_df)),
        success_rate=sr,
        median_alerts_per_day=float(per_pid["Alerts_per_day_median"].median()),
        mean_alerts_per_day=float(per_pid["Alerts_per_day_mean"].mean())
    )

sum_ml = summarize_alerts_dict(alerts_S1, name="ML")

# --- 2) Shuffled baseline: shuffle y_prob only, keep S1 constraints identical ---
def run_shuffled_baseline_S1(pdf, rng=42, **policy):
    shuffled = pdf.copy()
    rs = np.random.RandomState(rng)
    shuffled["y_prob"] = rs.permutation(shuffled["y_prob"].values)
    al = generate_alerts_with_policy_fast(shuffled, **policy)
    return al, summarize_alerts_dict(al, name="Shuffled")

alerts_shuf, sum_shuf = run_shuffled_baseline_S1(pred_df, **S1)

# --- 3) Random baseline: no probabilities; same S1 constraints (except high_conf) ---
def generate_alerts_random_same_constraints_S1(
    pdf: pd.DataFrame,
    min_hour: float = 8.0,
    max_hour: float = 22.0,
    require_two_hits: bool = True,
    two_hit_window_min: int = 45,
    cooldown_min: int = 90,
    per_day_cap: int = 5,
    seed: int = 42
):
    d = pdf.loc[(pdf["hour_float"] >= min_hour) & (pdf["hour_float"] <= max_hour),
                ["StudyID","ts","date","y_true"]].copy()
    if d.empty:
        return pd.DataFrame(columns=["StudyID","ts","date","y_true_at_alert","y_prob_at_alert","success"])

    rs = np.random.RandomState(seed)
    alerts = []
    win = np.timedelta64(two_hit_window_min, "m")
    cooldown = np.timedelta64(cooldown_min, "m")

    for (pid, day), g in d.sort_values(["StudyID","date","ts"]).groupby(["StudyID","date"]):
        times = g["ts"].to_numpy()
        yts   = g["y_true"].to_numpy(dtype=int)

        # require two-hit window if requested
        if require_two_hits:
            j_right = np.searchsorted(times, times + win, side="right") - 1
            has_pair = j_right > np.arange(len(times))
            has_pair &= (times[j_right] - times) <= win
            cand_idx = np.nonzero(has_pair)[0]
        else:
            cand_idx = np.arange(len(times))

        if len(cand_idx) == 0:
            continue

        order = rs.permutation(cand_idx)  # random order
        selected = []
        last_alert_time = times[0] - cooldown - np.timedelta64(1, "m")
        for i in order:
            if times[i] - last_alert_time < cooldown:
                continue
            selected.append(i)
            last_alert_time = times[i]
            if len(selected) >= per_day_cap:
                break

        for i in selected:
            alerts.append({
                "StudyID": pid, "ts": times[i], "date": day,
                "y_true_at_alert": int(yts[i]),
                "y_prob_at_alert": np.nan,
                "success": int(yts[i])
            })
    return pd.DataFrame(alerts)

def run_random_baseline_S1(pdf, **kwargs):
    al = generate_alerts_random_same_constraints_S1(pdf, **kwargs)
    return al, summarize_alerts_dict(al, name="Random")

alerts_rand, sum_rand = run_random_baseline_S1(
    pred_df,
    min_hour=S1["min_hour"],
    max_hour=S1["max_hour"],
    require_two_hits=S1["require_two_hits"],
    two_hit_window_min=S1["two_hit_window_min"],
    cooldown_min=S1["cooldown_min"],
    per_day_cap=S1["per_day_cap"],
    seed=123
)

# --- 4) Compare + lifts ---
compare = pd.DataFrame([sum_ml, sum_shuf, sum_rand])
print("\n=== S1 policy comparison ===")
print(compare)

def lift(a, b):
    if pd.isna(a) or pd.isna(b) or b == 0:
        return np.nan
    return a / b

print(f"\nLift vs Shuffled: {lift(sum_ml['success_rate'], sum_shuf['success_rate']):.2f}x")
print(f"Lift vs Random:   {lift(sum_ml['success_rate'], sum_rand['success_rate']):.2f}x")

# Optional: keep a handle for downstream analysis
current_alerts = alerts_S1



=== Policy-level alert summary (TABLE) ===
Total alerts: 632
Overall success rate: 0.131
Median alerts/day across participants: 1.50
Mean alerts/day across participants: 1.71

[ML/S1] Event-level: TP=83, FP=549, Total=632 (SR=0.131)

=== S1 policy comparison ===
       name  total_alerts  success_rate  median_alerts_per_day  \
0        ML           632      0.131329                    1.5   
1  Shuffled           791      0.083439                    2.0   
2    Random          1006      0.102386                    2.5   

   mean_alerts_per_day  
0             1.714025  
1             2.058470  
2             2.515216  

Lift vs Shuffled: 1.57x
Lift vs Random:   1.28x


# Clustering

In [145]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# =========================================
# Inputs (이미 세션에 있어야 함):
# - alerts_S1 : S1 정책 결과
# - pred_df    : OoF 메타정보
# - df_model   : 모델 학습에 쓴 피처 테이블
# - X_columns  : 피처 리스트
# =========================================

# 0) TP 알림만 분석
alerts_df = alerts_S1
assert {"StudyID","ts","success"}.issubset(alerts_df.columns), "alerts_S1 컬럼 확인 필요"
assert {"StudyID","ts","idx","y_prob","hour_float"}.issubset(pred_df.columns), "pred_df 컬럼 확인 필요"

tp_alerts = alerts_df.loc[alerts_df["success"] == 1].copy()
if tp_alerts.empty:
    raise ValueError("현재 정책에서 TP 알림이 없습니다.")

# 1) TP 알림 ↔ pred_df 조인
cols_to_merge = ["StudyID","ts","idx","y_prob","hour_float"]
tp_enriched = (
    tp_alerts.merge(pred_df[cols_to_merge],
                    on=["StudyID","ts"], how="left", validate="many_to_one")
    .dropna(subset=["idx"])
    .copy()
)
tp_enriched["idx"] = tp_enriched["idx"].astype(int)

# 2) 원본 피처 행렬
X_tp_full = df_model.loc[tp_enriched["idx"].values, X_columns].reset_index(drop=True)

TOP_FEATURES = [
    "Intensity","mean_intensity_1h","hungry_weighted","Time_24h",
    "15m_G_Diff_5","2_2.5h_pre_std","1_1.5h_pre_std","1.5_2h_pre_std",
    "15m_G_Diff_6","1_1.5h_pre_slope","15m_G_Diff_7","Z_Previous_4",
    "Sleep_Diff","TimeInBed","G_minus_T_pre_1.5h"
]
CLUSTER_FEATURES = [f for f in TOP_FEATURES if f in X_tp_full.columns]
if len(CLUSTER_FEATURES) == 0:
    CLUSTER_FEATURES = list(X_tp_full.columns)[:30]

X_tp = X_tp_full[CLUSTER_FEATURES].copy()
X_tp = X_tp.replace([np.inf, -np.inf], np.nan)
X_tp = X_tp.fillna(X_tp.median(numeric_only=True)).astype(float)

# 3) 스케일링
scaler = StandardScaler()
X_tp_scaled = scaler.fit_transform(X_tp)

# 4) k 선택 함수
def pick_k_and_fit(X, k_min=2, k_max=6, seed=42):
    n = len(X)
    if n < 2:
        km = KMeans(n_clusters=1, random_state=seed, n_init=10).fit(X)
        return 1, km, km.labels_, np.nan
    k_max_eff = min(k_max, max(k_min, n))
    best = None
    for k in range(k_min, k_max_eff + 1):
        km = KMeans(n_clusters=k, random_state=seed, n_init=10)
        labels = km.fit_predict(X)
        if len(np.unique(labels)) < 2:
            continue
        try:
            s = silhouette_score(X, labels)
        except Exception:
            s = np.nan
        if (best is None) or (np.isnan(best[3]) and not np.isnan(s)) or (not np.isnan(s) and s > best[3]):
            best = (k, km, labels, s)
    if best is None:
        km = KMeans(n_clusters=min(2, n), random_state=seed, n_init=10).fit(X)
        labels = km.labels_
        try:
            s = silhouette_score(X, labels) if len(np.unique(labels)) > 1 else np.nan
        except Exception:
            s = np.nan
        return min(2, n), km, labels, s
    return best

# ----- 이벤트(시점) 클러스터링 -----
k_evt, km_evt, labels_evt, sil_evt = pick_k_and_fit(X_tp_scaled)
tp_enriched["cluster_evt"] = labels_evt
sil_evt_str = "nan" if pd.isna(sil_evt) else f"{sil_evt:.3f}"
print(f"[Event-level] Best k={k_evt}, silhouette={sil_evt_str}, TP n={len(tp_enriched)}")

# 이벤트 클러스터 프로파일
feat_mean = X_tp.mean(axis=0)
feat_std  = X_tp.std(axis=0, ddof=0).replace(0, np.nan)

def profile_event_cluster(c, topn=10):
    mask = (tp_enriched["cluster_evt"] == c).values
    m = X_tp.loc[mask, :].mean(axis=0)
    z = (m - feat_mean) / feat_std
    return pd.DataFrame({"mean": m, "z": z}).sort_values("z", ascending=False).head(topn)

print("\n=== Event clusters: top +z features (top 10) ===")
for c in range(k_evt):
    n_ev = int((tp_enriched["cluster_evt"]==c).sum())
    print(f"\n[Cluster {c}] n_events={n_ev}")
    print(profile_event_cluster(c, topn=10).round(3))
# Time_24h 컬럼을 tp_enriched에 붙이기
tp_enriched = tp_enriched.reset_index(drop=True)
tp_enriched["Time_24h"] = X_tp["Time_24h"].values


evt_summary_time24 = (
    tp_enriched.groupby("cluster_evt")
    .agg(n_events=("idx","size"),
         n_participants=("StudyID","nunique"),
         mean_prob=("y_prob","mean"),
         median_prob=("y_prob","median"),
         mean_Time_24h=("Time_24h","mean"),
         median_Time_24h=("Time_24h","median"))
    .reset_index()
)

print("\n[Event-level] cluster summary (Time_24h 기반):\n", evt_summary_time24.round(3))


# PCA 2D 좌표
try:
    pca = PCA(n_components=2, random_state=42)
    X2 = pca.fit_transform(X_tp_scaled)
    tp_enriched["PC1"], tp_enriched["PC2"] = X2[:,0], X2[:,1]
    print("PCA explained variance (2D):", pca.explained_variance_ratio_.round(3))
except Exception as e:
    print("PCA skipped:", e)

# =========================================
# 참가자 클러스터링
# =========================================
tp_feats = pd.concat([tp_enriched.reset_index(drop=True), X_tp.reset_index(drop=True)], axis=1)

min_tp_per_pid = 2
pid_counts = tp_feats.groupby("StudyID")["idx"].size()
eligible_pids = pid_counts[pid_counts >= min_tp_per_pid].index
pid_mat = tp_feats[tp_feats["StudyID"].isin(eligible_pids)].groupby("StudyID")[CLUSTER_FEATURES].median()

pid_mat = pid_mat.replace([np.inf,-np.inf], np.nan)
pid_mat = pid_mat.fillna(pid_mat.median(numeric_only=True)).astype(float)

scaler_pid = StandardScaler()
X_pid_scaled = scaler_pid.fit_transform(pid_mat)

k_pid, km_pid, labels_pid, sil_pid = pick_k_and_fit(X_pid_scaled)
pid_mat = pid_mat.copy()
pid_mat["cluster_pid"] = labels_pid
sil_pid_str = "nan" if pd.isna(sil_pid) else f"{sil_pid:.3f}"
print(f"\n[Participant-level] Best k={k_pid}, silhouette={sil_pid_str}, participants n={pid_mat.shape[0]}")

def profile_pid_cluster(c, topn=10):
    Xc = pid_mat.loc[pid_mat["cluster_pid"] == c, CLUSTER_FEATURES]
    z = (Xc.mean(axis=0) - pid_mat[CLUSTER_FEATURES].mean(axis=0)) / pid_mat[CLUSTER_FEATURES].std(axis=0, ddof=0).replace(0, np.nan)
    return pd.DataFrame({"mean": Xc.mean(axis=0), "z": z}).sort_values("z", ascending=False).head(topn)

print("\n=== Participant clusters: top +z features (top 10) ===")
for c in range(k_pid):
    n_pid = int((pid_mat['cluster_pid']==c).sum())
    print(f"\n[PID Cluster {c}] n_participants={n_pid}")
    print(profile_pid_cluster(c, topn=10).round(3))

pid_tp_counts = (
    tp_enriched.merge(pid_mat[["cluster_pid"]], left_on="StudyID", right_index=True, how="left")
    .groupby("cluster_pid")["idx"].size().rename("TP_events").reset_index()
)
print("\n[Participant-level] cluster -> TP events count:\n", pid_tp_counts)


[Event-level] Best k=2, silhouette=0.234, TP n=83

=== Event clusters: top +z features (top 10) ===

[Cluster 0] n_events=59
                       mean      z
hungry_weighted       0.173  0.147
Intensity             0.542  0.099
mean_intensity_1h     0.424  0.047
2_2.5h_pre_std        3.739  0.025
Sleep_Diff          912.136  0.020
G_minus_T_pre_1.5h  -18.318 -0.068
TimeInBed           371.797 -0.122
Time_24h             15.578 -0.226
15m_G_Diff_5          1.661 -0.264
1.5_2h_pre_std        2.181 -0.313

[Cluster 1] n_events=24
                       mean      z
15m_G_Diff_6         14.500  1.273
1_1.5h_pre_slope      0.967  1.273
1_1.5h_pre_std       10.253  1.271
15m_G_Diff_7         11.625  1.024
Z_Previous_4          2.035  0.949
1.5_2h_pre_std        6.187  0.769
15m_G_Diff_5          9.042  0.650
Time_24h             19.501  0.555
TimeInBed           433.208  0.299
G_minus_T_pre_1.5h  -13.904  0.168

[Event-level] cluster summary (Time_24h 기반):
    cluster_evt  n_events  n_parti

# Key Interpretation

## Event-level
- **Cluster 0**: Occurs around 3:30 PM; low glucose variability; slightly higher hunger and sedentary.
- **Cluster 1**: Concentrated after 7:30 PM; very high glucose variability and spike magnitude; higher pre-event glucose levels.

## Participant-level
- **Cluster 0**: Participants with low glucose variability, slightly higher hunger and sedentary.
- **Cluster 1**: Participants with events concentrated in the evening, showing high glucose variability and elevated pre-event glucose levels.


# SHAP values

In [19]:
import shap
# isnull 변수 제외
cols_no_isnull = [c for c in X_columns if not c.endswith("_isnull")]

# SHAP 값 데이터프레임 생성 (절대값)
abs_shap = np.abs(shap_values)
shap_df = pd.DataFrame(abs_shap, columns=X_columns)

# isnull 제외한 데이터프레임
shap_df_filtered = shap_df[cols_no_isnull]

# Threshold 적용해서 빈도 계산
threshold_shap = 0.01
freq = (shap_df_filtered > threshold_shap).sum().sort_values(ascending=False)
mean_abs = shap_df_filtered.mean().sort_values(ascending=False)
mode_abs = shap_df_filtered.mode().iloc[0]



In [21]:
# 1) isnull 변수 제거
exclude_cols = [col for col in X_columns if col.endswith("_isnull")]
freq_filtered = freq.drop(index=exclude_cols, errors="ignore")
mean_abs_filtered = mean_abs.drop(index=exclude_cols, errors="ignore")

# 2) 두 개 시리즈를 하나의 DataFrame으로 병합
shap_summary = pd.DataFrame({
    "Frequency": freq_filtered,
    "Mean_Effect": mean_abs_filtered
})

# 3) Frequency 기준 내림차순 정렬
shap_summary = shap_summary.sort_values(by="Frequency", ascending=False)

# 4) 결과 확인
print(shap_summary.head(15))


                    Frequency  Mean_Effect
hungry_weighted           280     0.061343
Intensity                 278     0.090791
mean_intensity_1h         263     0.039572
Time_24h                  213     0.025629
15m_G_Diff_5              180     0.022950
G_minus_T_pre_1.5h        165     0.016370
2_2.5h_pre_std            145     0.020789
15m_G_Diff_7              128     0.014066
1_1.5h_pre_std             87     0.009001
Z_HR                       77     0.009343
Z_Previous_4               76     0.008751
EMA_T_Diff                 70     0.009307
1.5_2h_pre_std             63     0.006509
Z_Intensity                56     0.006421
Efficiency                 50     0.005929
